In [1]:
# builtins
import sys
import os
import time
import logging
from datetime import timedelta
from logging.config import dictConfig
import numpy as np
import datetime
import pathlib
import pandas as pd
import joblib

# externals
import xarray as xr

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# locals
from downscaleml.core.dataset import ERA5Dataset, NetCDFDataset, EoDataset

from downscaleml.main.config import (NET, ERA5_PLEVELS, ERA5_PREDICTORS, PREDICTAND,
                                     CALIB_PERIOD, VALID_PERIOD, DOY, NORM,
                                     OVERWRITE, DEM, DEM_FEATURES, STRATIFY,
                                     WET_DAY_THRESHOLD, VALID_SIZE, 
                                     start_year, end_year, CHUNKS)

from downscaleml.main.inputoutput import (ERA5_PATH, OBS_PATH, DEM_PATH, MODEL_PATH, TARGET_PATH)

from downscaleml.core.constants import (ERA5_P_VARIABLES, ERA5_P_VARIABLES_SHORTCUT, ERA5_P_VARIABLE_NAME,
                                        ERA5_S_VARIABLES, ERA5_S_VARIABLES_SHORTCUT, ERA5_S_VARIABLE_NAME,
                                        ERA5_VARIABLES, ERA5_VARIABLE_NAMES, ERA5_PRESSURE_LEVELS,
                                        PREDICTANDS, ERA5_P_VARIABLES, ERA5_S_VARIABLES)

from downscaleml.core.utils import NAMING_Model, normalize, search_files, LogConfig
from downscaleml.core.logging import log_conf
    
# module level logger
LOGGER = logging.getLogger(__name__)

def stacker(xarray_dataset):
    # stack along the lat and lon dimensions
    stacked = xarray_dataset.stack()
    dask_arr = stacked.to_array().data
    xarray_dataset = dask_arr.T
    LogConfig.init_log('Shape of the numpy array is in (spatial, time, variables):{}'.format(xarray_dataset.shape))
    return xarray_dataset

def doy_encoding(X, y=None, doy=False):

    # whether to include the day of the year as predictor variable
    if doy:
        # add doy to set of predictor variables
        LOGGER.info('Adding day of the year to predictor variables ...')
        X = X.assign(EoDataset.encode_doys(X, chunks=X.chunks))

    print(X)
    return X

if __name__ == '__main__':

    # initialize timing
    start_time = time.monotonic()
        
    # initialize network filename
    state_file = NAMING_Model.state_file(
        NET, PREDICTAND, ERA5_PREDICTORS, ERA5_PLEVELS, WET_DAY_THRESHOLD, dem=DEM,
        dem_features=DEM_FEATURES, doy=DOY, stratify=STRATIFY)
    
    state_file = MODEL_PATH.joinpath(PREDICTAND, state_file)
    target = TARGET_PATH.joinpath(PREDICTAND)

    # check if output path exists
    if not target.exists():
        target.mkdir(parents=True, exist_ok=True)
    # initialize logging
    log_file = state_file.with_name(state_file.name + "_log.txt")
    
    if log_file.exists():
        log_file.unlink()
    dictConfig(log_conf(log_file))

    # check if target dataset already exists
    target = target.joinpath(state_file.name + '.nc')
    if target.exists() and not OVERWRITE:
        LogConfig.init_log('{} already exists.'.format(target))
        sys.exit()

    LogConfig.init_log('Initializing downscaling for period: {}'.format(
        ' - '.join([str(CALIB_PERIOD[0]), str(CALIB_PERIOD[-1])])))

    # initialize ERA5 predictor dataset
    LogConfig.init_log('Initializing ERA5 predictors.')
    Era5 = ERA5Dataset(ERA5_PATH.joinpath('ERA5'), ERA5_PREDICTORS,
                       plevels=ERA5_PLEVELS)
    Era5_ds = Era5.merge(chunks=CHUNKS)
    Era5_ds = Era5_ds.rename({'lon': 'x','lat': 'y'})
    
    # initialize OBS predictand dataset
    LogConfig.init_log('Initializing observations for predictand: {}'
                       .format(PREDICTAND))

    # read in-situ gridded observations
    Obs_ds = search_files(OBS_PATH.joinpath(PREDICTAND), '.nc$').pop()
    Obs_ds = xr.open_dataset(Obs_ds)
    Obs_ds = Obs_ds.rename({'lon': 'x','lat': 'y'})

    # whether to use digital elevation model
    if DEM:
        # digital elevation model: Copernicus EU-Dem v1.1
        dem = search_files(DEM_PATH, '^interTwin_dem.nc$').pop()

        # read elevation and compute slope and aspect
        dem = ERA5Dataset.dem_features(
            dem, {'y': Era5_ds.y, 'x': Era5_ds.x},
            add_coord={'time': Era5_ds.time})

        # check whether to use slope and aspect
        if not DEM_FEATURES:
            dem = dem.drop_vars(['slope', 'aspect']).chunk(Era5_ds.chunks)

        # add dem to set of predictor variables
        dem = dem.chunk(Era5_ds.chunks)
        Era5_ds = xr.merge([Era5_ds, dem])

    # initialize training data
    LogConfig.init_log('Initializing training data.')

    # split calibration period into training and validation period
    if PREDICTAND == 'pr' and STRATIFY:
        # stratify training and validation dataset by number of
        # observed wet days for precipitation
        wet_days = (Obs_ds.sel(time=CALIB_PERIOD).mean(dim=('y', 'x'))
                    >= WET_DAY_THRESHOLD).to_array().values.squeeze()
        train, valid = train_test_split(
            CALIB_PERIOD, stratify=wet_days, test_size=VALID_SIZE)

        # sort chronologically
        train, valid = sorted(train), sorted(valid)
        Era5_train, Obs_train = Era5_ds.sel(time=train), Obs_ds.sel(time=train)
        Era5_valid, Obs_valid = Era5_ds.sel(time=valid), Obs_ds.sel(time=valid)
    else:
        LogConfig.init_log('We are not calculating Stratified Precipitation based on Wet Days here!')

    # training and validation dataset
    Era5_train, Obs_train = Era5_ds.sel(time=CALIB_PERIOD), Obs_ds.sel(time=CALIB_PERIOD)
    Era5_valid, Obs_valid = Era5_ds.sel(time=VALID_PERIOD), Obs_ds.sel(time=VALID_PERIOD)

    Era5_train = doy_encoding(Era5_train, Obs_train, doy=DOY)
    Era5_valid = doy_encoding(Era5_valid, Obs_valid, doy=DOY)

    predictors_train = Era5_train
    predictors_valid = Era5_valid
    predictand_train = Obs_train
    predictand_valid = Obs_valid
    
    predictors_train = stacker(predictors_train).compute()
    predictors_valid = stacker(predictors_valid).compute()
    predictand_train = stacker(predictand_train)
    predictand_valid = stacker(predictand_valid)
    
    LogConfig.init_log('Dask computations done!')
    # iterate over the grid points
    LogConfig.init_log('Downscaling by Random Forest Starts: iterating each grid cell over time dimension')
    
    Models = {
        'RandomForestRegressor' : RandomForestRegressor,
        'XGBRegressor' : XGBRegressor,
        'AdaBoostRegressor': AdaBoostRegressor,
        'LGBMRegressor': LGBMRegressor,
    }
    Model_name = NET

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:51:50: Initializing downscaling for period: 1985-01-01 - 2015-12-31
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:51:50: Initializing ERA5 predictors.
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: Searching: /mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/hydroModelDownscale/p_REANALYSIS/ERA5/geopotential, pattern: .nc$
downscaleml.core.utils: Searching: /mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/hydroModelDownscale/p_REANALYSIS/ERA5/temperature, pattern: .nc$
downscaleml.core.utils: Searching: /mnt/CEPH_PROJECTS/InterTwin/Climate_Downscaling/hydroModelDownsc

<xarray.Dataset>
Dimensions:    (time: 11322, x: 161, y: 96)
Coordinates:
  * time       (time) datetime64[ns] 1985-01-01 1985-01-02 ... 2015-12-31
  * x          (x) float64 5.084 5.151 5.218 5.285 ... 15.62 15.69 15.76 15.82
  * y          (y) float64 43.62 43.69 43.75 43.82 ... 49.8 49.86 49.93 50.0
Data variables: (12/17)
    z_500      (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    z_850      (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    t_500      (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    t_850      (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    u_500      (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    u_850      (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    ...         ...
    t2m        (time, y, x) float32 dask.array<chunksize=(364, 96, 161), meta=np.ndarray>
    elevation  (time, 

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:52:06: Shape of the numpy array is in (spatial, time, variables):(161, 96, 11322, 17)
downscaleml.core.utils: --------------------------------------------------------------------------------


<xarray.Dataset>
Dimensions:    (time: 1826, x: 161, y: 96)
Coordinates:
  * time       (time) datetime64[ns] 2016-01-01 2016-01-02 ... 2020-12-30
  * x          (x) float64 5.084 5.151 5.218 5.285 ... 15.62 15.69 15.76 15.82
  * y          (y) float64 43.62 43.69 43.75 43.82 ... 49.8 49.86 49.93 50.0
Data variables: (12/17)
    z_500      (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    z_850      (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    t_500      (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    t_850      (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    u_500      (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    u_850      (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    ...         ...
    t2m        (time, y, x) float32 dask.array<chunksize=(357, 96, 161), meta=np.ndarray>
    elevation  (time, y

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:14: Shape of the numpy array is in (spatial, time, variables):(161, 96, 1826, 17)
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:57: Shape of the numpy array is in (spatial, time, variables):(161, 96, 11322, 1)
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:58: Shape of the numpy array is in (spatial, time, variables):(161, 96, 1826, 1)
downscaleml.core.utils: --------------------------------------------------------------------------------
do

CV Steps - 10 fold cross validation

In [2]:
import math
import random

random.seed(42)

def grid_creator(combinations, numpy_object):
    gridded = np.ones(shape=(int(math.sqrt(combinations)), int(math.sqrt(combinations)), numpy_object.shape[2], numpy_object.shape[3])) * np.nan
    return gridded

combination = 100

predictors_train_grid = grid_creator(combination, predictors_train)
predictand_train_grid = grid_creator(combination, predictand_train)
predictors_valid_grid = grid_creator(combination, predictors_valid)
predictand_valid_grid = grid_creator(combination, predictand_valid)

x_range = (0, (predictand_train.shape[0] - 1))
y_range = (0, (predictand_train.shape[1] - 1))

x_numpy = np.ones(shape=(int(math.sqrt(combination)), int(math.sqrt(combination))))
y_numpy = np.ones(shape=(int(math.sqrt(combination)), int(math.sqrt(combination))))

for i in range(int(math.sqrt(combination))):
    for j in range(int(math.sqrt(combination))):
        x = random.randint(*x_range)
        y = random.randint(*y_range)
        x_numpy[i][j] = int(x)
        y_numpy[i][j] = int(y)

x = x_numpy.astype(int)
y = y_numpy.astype(int)

for i in range(int(math.sqrt(combination))):
    for j in range(int(math.sqrt(combination))):
        predictors_train_grid[i][j] = predictors_train[x[i][j], y[i][j], :, :]
        predictand_train_grid[i][j] = predictand_train[x[i][j], y[i][j], :, :]
        predictors_valid_grid[i][j] = predictors_valid[x[i][j], y[i][j], :, :] 
        predictand_valid_grid[i][j] = predictand_valid[x[i][j], y[i][j], :, :] 



Single Model - Iterating over all the grids

In [3]:
prediction = np.ones(shape=(predictand_valid_grid.shape[2], predictand_valid_grid.shape[1], predictand_valid_grid.shape[0])) * np.nan
model = Models[Model_name]()
r2_values = np.ones(shape=(int(math.sqrt(combination)), int(math.sqrt(combination))))
mae_values = np.ones(shape=(int(math.sqrt(combination)), int(math.sqrt(combination))))

for i in range(predictors_train_grid.shape[0]):
    for j in range(predictors_train_grid.shape[1]):

        point_predictors = predictors_train_grid[i, j, :, :]
        point_predictand = predictand_train_grid[i, j, :, :]

        # check if the grid point is valid
        if np.isnan(point_predictors).any() or np.isnan(point_predictand).any():
            # move on to next grid point
            continue

        # prepare predictors of validation period
        point_validation = predictors_valid_grid[i, j, :, :]
        #point_validation = normalize(point_validation)

        predictand_validation = predictand_valid_grid[i, j, :, :]

        LogConfig.init_log('Current grid point: ({:d}), ({:d}) '.format(i, j))    
        # normalize each predictor variable to [0, 1]
        # point_predictors = normalize(point_predictors)
        # instanciate the model for the current grid point

        # train model on training data
        model.fit(point_predictors, point_predictand)
        # predict validation period
        pred = model.predict(point_validation)
        
        LogConfig.init_log('Processing grid point: {:d}, {:d} - score: {:.2f}'.format(i, j, r2_score(predictand_validation, pred)))
        r2_values[i][j] = r2_score(predictand_validation, pred)
        LogConfig.init_log('Processing grid point: {:d}, {:d} - score: {:.2f}'.format(i, j, mean_absolute_error(predictand_validation, pred)))
        mae_values[i][j] = mean_absolute_error(predictand_validation, pred)
        # store predictions for current grid point
        prediction[:, j, i] = pred

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:58: Current grid point: (0), (0) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:58: Processing grid point: 0, 0 - score: 0.98
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.util

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000889 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 281.892061
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000310 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 285.330531
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000295 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:58: Processing grid point: 0, 2 - score: 0.98
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:58: Processing grid point: 0, 2 - score: 0.90
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:58: Current grid point: (0), (3) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000353 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 283.062973
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 289.645019
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000312 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:59: Processing grid point: 0, 5 - score: 0.99
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:59: Processing grid point: 0, 5 - score: 0.64
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:55:59: Current grid point: (0), (6) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 283.009566
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000308 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 282.271507
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000310 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:07: Processing grid point: 9, 5 - score: 0.97
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:07: Processing grid point: 9, 5 - score: 0.87
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:07: Current grid point: (9), (6) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000385 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 279.801890
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000307 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 287.767225
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000290 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:07: Processing grid point: 9, 8 - score: 0.99
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:07: Processing grid point: 9, 8 - score: 0.69
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:07: Current grid point: (9), (9) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000298 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 283.987753


Multiple Models 

In [4]:
prediction = np.ones(shape=(predictand_valid_grid.shape[2], predictand_valid_grid.shape[1], predictand_valid_grid.shape[0])) * np.nan

r2_values_m = np.ones(shape=(int(math.sqrt(combination)), int(math.sqrt(combination))))
mae_values_m = np.ones(shape=(int(math.sqrt(combination)), int(math.sqrt(combination))))
models_array = np.empty(shape=((int(math.sqrt(combination))), (int(math.sqrt(combination)))), dtype=object)

for i in range(predictors_train_grid.shape[0]):
    for j in range(predictors_train_grid.shape[1]):

        point_predictors = predictors_train_grid[i, j, :, :]
        point_predictand = predictand_train_grid[i, j, :, :]

        # check if the grid point is valid
        if np.isnan(point_predictors).any() or np.isnan(point_predictand).any():
            # move on to next grid point
            continue

        # prepare predictors of validation period
        point_validation = predictors_valid_grid[i, j, :, :]
        #point_validation = normalize(point_validation)

        predictand_validation = predictand_valid_grid[i, j, :, :]

        LogConfig.init_log('Current grid point: ({:d}), ({:d}) '.format(i, j))    
        # normalize each predictor variable to [0, 1]
        # point_predictors = normalize(point_predictors)
        # instanciate the model for the current grid point
        
        model = Models[Model_name]()

        # train model on training data
        model.fit(point_predictors, point_predictand)
        # predict validation period
        pred = model.predict(point_validation)
        models_array[i][j] = model
        
        LogConfig.init_log('Processing grid point: {:d}, {:d} - score: {:.2f}'.format(i, j, r2_score(predictand_validation, pred)))
        r2_values_m[i][j] = r2_score(predictand_validation, pred)
        LogConfig.init_log('Processing grid point: {:d}, {:d} - score: {:.2f}'.format(i, j, mean_absolute_error(predictand_validation, pred)))
        mae_values_m[i][j] = mean_absolute_error(predictand_validation, pred)

        # store predictions for current grid point
        prediction[:, j, i] = pred

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Current grid point: (0), (0) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Processing grid point: 0, 0 - score: 0.98
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.util

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000302 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 281.892061
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 285.330531


downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Processing grid point: 0, 1 - score: 0.99
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Processing grid point: 0, 1 - score: 0.73
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Current grid point: (0), (2) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000326 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 284.977385


downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Processing grid point: 0, 2 - score: 0.98
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Processing grid point: 0, 2 - score: 0.90
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:08: Current grid point: (0), (3) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000317 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 282.271507
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 274.394897
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000293 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start

downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:17: Processing grid point: 9, 6 - score: 0.99
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:17: Processing grid point: 9, 6 - score: 0.70
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:17: Current grid point: (9), (7) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000305 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 287.767225
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 281.904987


downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:17: Processing grid point: 9, 8 - score: 0.99
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:17: Processing grid point: 9, 8 - score: 0.69
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:17: Current grid point: (9), (9) 
downscaleml.core.utils: --------------------------------------------------------------------------------
/home/sdhinakaran/micromamba/envs/check/lib/python3.11/site-packages/sklearn/utils/validation.py:1183: DataConversionWarn

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000306 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3499
[LightGBM] [Info] Number of data points in the train set: 11322, number of used features: 14
[LightGBM] [Info] Start training from score 283.987753


In [5]:
r2_single = np.mean(r2_values)
mae_single = np.mean(mae_values)
LogConfig.init_log('Single model iteration over the entire grid R2 Score:{} and MAE:{}'.format(r2_single, mae_single))


r2_multi = np.mean(r2_values_m)
mae_multi = np.mean(mae_values_m)
LogConfig.init_log('Multiple model iteration over the each grid R2 Score:{} and MAE:{}'.format(r2_multi, mae_multi))



downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:18: Single model iteration over the entire grid R2 Score:0.9845628283956361 and MAE:0.7029637817507424
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: --------------------------------------------------------------------------------
downscaleml.core.utils: 2024-02-15T14:56:18: Multiple model iteration over the each grid R2 Score:0.9845628283956361 and MAE:0.7029637817507424
downscaleml.core.utils: --------------------------------------------------------------------------------
